# Prueba de Cero Conocimiento a partir del MPC SDitH

En este notebook vamos a crear una prueba de cero conocimiento a partir del protocolo MPC SDitH. Para hacer esto, tomamos el protocolo MPC SDitH y usando la idea de computación multiparte "en la cabeza" vamos a crear la prueba de cero conocimiento interactiva.

En este notebook vamos a crear una prueba de cero conocimiento a partir del protocolo MPC SDitH. Para hacer esto, tomamos el protocolo MPC SDitH y usamos la idea de computación multipartita "en la cabeza" para crear una prueba de cero conocimiento interactiva.

Este protocolo se describe en la siguiente secuencia de pasos:

- El probador genera las entradas compartidas aleatorias $[P], [Q], [a], [b], [c]$ y se compromete (_commit_) con las particiones de las partes:
  $[P]_i, [Q]_i, [a]_i, [b]_i, [c]_i \rightarrow \text{com}_i$, donde $i$ representa a cada una de las partes.

- El verificador desafía al probador con un valor de aleatoriedad $r, \epsilon$ para el protocolo MPC.

- El probador ejecuta el protocolo MPC "en su cabeza" utilizando las entradas compartidas $[P], [Q], [a], [b], [c]$, junto con la aleatoriedad $r, \epsilon$, y obtiene los valores de difusión $[\alpha], [\beta], [v]$.

- El verificador desafía al probador a abrir todas las partes en el conjunto $I \subset [1:N]$, donde $I$ es el índice de las partes que se deben revelar.

- El probador revela las comparticiones de entrada de todas las partes en el conjunto $I$: $[P]_i, [Q]_i, [a]_i, [b]_i, [c]_i$ para $i \in I$, junto con los compromisos $\text{com}_i$. También revela los valores de difusión $[\alpha], [\beta], [v]$.

- El verificador verifica la consistencia de los compromisos y de la computación MPC para las partes reveladas. Si todos los valores son consistentes, el verificador acepta la prueba.


A continuación, vamos a hacer el proceso.

## 1. Configuración inicial 
##### (Probador)

En este caso, el objetivo de nuestra prueba de cero conocimiento es demostrar que conocemos una solución para el problema de decodificación del síndrome. Para esto se hace uso de tres polinomios $S,Q,P$ asociados a nuestra solución $x$ y un polinomio público $F$. Estas serán las entradas de cada una de las partes involucradas en el protocolo MPC.

En esta sección creamos los polinomios y creamos las particiones correspondientes.

In [1]:
import random
import hashlib

# Definir parámetros
definir_campo_q = 2  # Campo finito para el problema de decodificación del síndrome
F_q = GF(definir_campo_q)  # Campo finito de orden q
n = 6  # Longitud del vector de error
k = 3  # Longitud del vector de mensaje
m = n - k  # Número de filas de la matriz de paridad H

# Campo finito para los polinomios (debe ser mayor que F_q)
definir_campo_p = 17  # Campo finito para los polinomios, debe ser mayor que F_q y mayor que n
F_p = GF(definir_campo_p)  # Campo finito de orden p

# Definir el vector x en el campo F_q de longitud n
x = vector(F_q, [0, 0, 0, 0, 1, 1])  # Ejemplo con valores específicos

# Asociar los primeros n elementos del campo F_p con las coordenadas de x
f = [F_p(i) for i in range(n)]  # f_i para i en [1:n]

# Polinomio S: Interpolación de Lagrange usando los puntos (f_i, x_i)
R = F_p['X']
S = R.lagrange_polynomial([(f[i], x[i]) for i in range(n)])

# Polinomio Q: Producto de (X - f_i) para i en el subconjunto de índices con valor x_i = 1
X = R.gen()
E = [i for i in range(n) if x[i] != 0]  # Subconjunto de índices no nulos de x
Q = prod([(X - f[i]) for i in E])

# Polinomio F: Polinomio de "desaparición" para todos los f_i
F = prod([(X - f[i]) for i in range(n)])

# Polinomio P: Definido como P = S * Q / F
P = (S * Q).quo_rem(F)[0]  # División polinomial asegurando que F divide a S * Q sin residuo

# Evaluar los polinomios S y Q en los puntos f
evaluaciones_S = [S(f_i) for f_i in f]
evaluaciones_Q = [Q(f_i) for f_i in f]

# Imprimir los polinomios
def imprimir_polinomios():
    print("Polinomio S (Interpolación de Lagrange):")
    print(S)
    print("\nEvaluaciones de S en f:")
    print(evaluaciones_S)
    print("\nSubconjunto E:")
    print(E)
    print("\nPolinomio Q (Producto de raíces para índices no nulos de x):")
    print(Q)
    print("\nEvaluaciones de Q en f:")
    print(evaluaciones_Q)
    print("\nPolinomio F (Vanishing Polynomial):")
    print(F)
    print("\nPolinomio P (Resultado de S * Q / F):")
    print(P)
    
    print("\nEvaluación de Polinomios")
    print(f"\nS*Q==P*F: {S*Q==P*F}")

imprimir_polinomios()

Polinomio S (Interpolación de Lagrange):
13*X^5 + 11*X^4 + 10*X

Evaluaciones de S en f:
[0, 0, 0, 0, 1, 1]

Subconjunto E:
[4, 5]

Polinomio Q (Producto de raíces para índices no nulos de x):
X^2 + 8*X + 3

Evaluaciones de Q en f:
[3, 12, 6, 2, 0, 0]

Polinomio F (Vanishing Polynomial):
X^6 + 2*X^5 + 13*X^3 + 2*X^2 + 16*X

Polinomio P (Resultado de S * Q / F):
13*X + 4

Evaluación de Polinomios

S*Q==P*F: True


In [2]:
# Dividimos los polinomios S, Q y P en partes
n_parts = 5  # Número de partes

def dividir_polinomio_en_partes(polinomio, n_parts, campo):
    coeficientes = polinomio.coefficients(sparse=False)
    grado = polinomio.degree()
    partes = [[] for _ in range(n_parts)]  # Crear listas para cada parte

    for i in range(grado + 1):
        # Generamos aleatoriamente las primeras (n_parts - 1) partes en el campo
        partial_shares = [campo.random_element() for _ in range(n_parts - 1)]
        # Calculamos la última parte de manera que la suma de todas las partes sea igual al coeficiente original
        last_share = coeficientes[i] - sum(partial_shares)
        # Añadimos la última parte a la lista de partes
        partial_shares.append(last_share)
        # Distribuimos cada parte correspondiente entre las listas de partes de los participantes
        for j in range(n_parts):
            partes[j].append(partial_shares[j])
        
        # Imprimir las partes generadas para el coeficiente actual
        print(f"Coeficiente de x^{i} ({coeficientes[i]}): Partes -> {partial_shares}")

    return partes

# Función para generar los polinomios a partir de las partes
def construir_polinomios_de_partes(shares, n_parts, X):
    polinomios = []
    for j in range(n_parts):
        part_poly = sum(shares[j][i] * X^i for i in range(len(shares[j])))
        polinomios.append(part_poly)
    return polinomios

# Dividimos los polinomios S, Q y P en partes
print("\nDividiendo el polinomio S en partes:")
shares_S = dividir_polinomio_en_partes(S, n_parts, F_p)

print("\nDividiendo el polinomio Q en partes:")
shares_Q = dividir_polinomio_en_partes(Q, n_parts, F_p)

print("\nDividiendo el polinomio P en partes:")
shares_P = dividir_polinomio_en_partes(P, n_parts, F_p)

# Construimos los polinomios para cada una de las partes
def imprimir_polinomios_partes():
    partes_S_polinomios = construir_polinomios_de_partes(shares_S, n_parts, X)
    partes_Q_polinomios = construir_polinomios_de_partes(shares_Q, n_parts, X)
    partes_P_polinomios = construir_polinomios_de_partes(shares_P, n_parts, X)

    print("\nPolinomio que recibe cada parte para el polinomio S:")
    for j, part_poly in enumerate(partes_S_polinomios):
        print(f"Parte {j + 1} de S: {part_poly}")

    print("\nPolinomio que recibe cada parte para el polinomio Q:")
    for j, part_poly in enumerate(partes_Q_polinomios):
        print(f"Parte {j + 1} de Q: {part_poly}")

    print("\nPolinomio que recibe cada parte para el polinomio P:")
    for j, part_poly in enumerate(partes_P_polinomios):
        print(f"Parte {j + 1} de P: {part_poly}")

imprimir_polinomios_partes()

# Función para generar valores aleatorios en el campo
def generar_valores_aleatorios(campo, n_parts):
    valores = [campo.random_element() for _ in range(n_parts)]
    print(f"Valores aleatorios generados: {valores}")
    return valores

# Generar valores aleatorios para a_shares y b_shares
print("\nGenerando partes para a y b:")
a_shares = generar_valores_aleatorios(F_p, n_parts)
b_shares = generar_valores_aleatorios(F_p, n_parts)

# Calcular el producto c = a * b, y luego compartir el valor c
c = sum(a_shares) * sum(b_shares)
print(f"\nValor de c (producto de a y b): {c}")

# Generar partes para c_shares
c_shares = generar_valores_aleatorios(F_p, n_parts)
c_sum = sum(c_shares)
adjustment = c - c_sum
print(f"\nSuma de las partes de c antes del ajuste: {c_sum}")
print(f"Ajuste necesario para c: {adjustment}")

# Ajustar la primera parte de c_shares para garantizar que sum(c_shares) == c
c_shares[0] += adjustment
print(f"Partes ajustadas de c: {c_shares}")

print("\nResumen de lo que recibe cada parte:")
for i in range(n_parts):
    print(f"Parte {i + 1} recibe:")
    print(f"  - Coeficientes de S: {shares_S[i]}")
    print(f"  - Coeficientes de Q: {shares_Q[i]}")
    print(f"  - Coeficientes de P: {shares_P[i]}")
    print(f"  - Parte de a: {a_shares[i]}")
    print(f"  - Parte de b: {b_shares[i]}")
    print(f"  - Parte de c: {c_shares[i]}")


Dividiendo el polinomio S en partes:
Coeficiente de x^0 (0): Partes -> [15, 7, 8, 6, 15]
Coeficiente de x^1 (10): Partes -> [16, 11, 3, 3, 11]
Coeficiente de x^2 (0): Partes -> [14, 16, 3, 9, 9]
Coeficiente de x^3 (0): Partes -> [13, 3, 14, 11, 10]
Coeficiente de x^4 (11): Partes -> [12, 0, 12, 0, 4]
Coeficiente de x^5 (13): Partes -> [12, 0, 1, 12, 5]

Dividiendo el polinomio Q en partes:
Coeficiente de x^0 (3): Partes -> [7, 7, 14, 4, 5]
Coeficiente de x^1 (8): Partes -> [14, 13, 5, 4, 6]
Coeficiente de x^2 (1): Partes -> [1, 5, 15, 10, 4]

Dividiendo el polinomio P en partes:
Coeficiente de x^0 (4): Partes -> [0, 14, 9, 11, 4]
Coeficiente de x^1 (13): Partes -> [0, 15, 16, 1, 15]

Polinomio que recibe cada parte para el polinomio S:
Parte 1 de S: 12*X^5 + 12*X^4 + 13*X^3 + 14*X^2 + 16*X + 15
Parte 2 de S: 3*X^3 + 16*X^2 + 11*X + 7
Parte 3 de S: X^5 + 12*X^4 + 14*X^3 + 3*X^2 + 3*X + 8
Parte 4 de S: 12*X^5 + 11*X^3 + 9*X^2 + 3*X + 6
Parte 5 de S: 5*X^5 + 4*X^4 + 10*X^3 + 9*X^2 + 11*X

### Envío de compromisos

Ya que se establecieron las entradas de cada parte del protocolo MPC que va a ejecutar el probador, se realiza un compromiso (_commit_) de cada una de las entradas y se envía el resultado de este compromiso al verificador.

In [3]:
# Función para obtener el commit de cada parte usando una función hash
def obtener_commit_de_parte_hash(shares_S, shares_Q, shares_P, a_shares, b_shares, c_shares, campo):
    commits = []
    valores_aleatorios = []
    for i in range(n_parts):
        # Generar un valor aleatorio para cada parte
        valor_aleatorio = campo.random_element()
        valores_aleatorios.append(valor_aleatorio)
        
        # Crear una cadena con todos los valores para la parte i
        datos_str = (
            f"{shares_S[i]}{shares_Q[i]}{shares_P[i]}"
            f"{a_shares[i]}{b_shares[i]}{c_shares[i]}"
            f"{valor_aleatorio}"
        )

        # Calcular el hash usando SHA-256
        commit = hashlib.sha256(datos_str.encode()).hexdigest()
        commits.append(commit)
        print(f"Commit de la parte {i + 1}: {commit} (valor aleatorio utilizado: {valor_aleatorio})")

    return commits, valores_aleatorios

# Obtener los commits de cada parte usando la función hash
print("\nObteniendo los commits de cada parte usando hash:")
commits, valores_aleatorios = obtener_commit_de_parte_hash(shares_S, shares_Q, shares_P, a_shares, b_shares, c_shares, F_p)


Obteniendo los commits de cada parte usando hash:
Commit de la parte 1: 8e73a2687e2d1f34ee89aec745df6020ce2a31205d0e955b97bcdf33207f135e (valor aleatorio utilizado: 12)
Commit de la parte 2: bebe5b1703ec288e208f0f4450e123b7cea261f96a5fbc021a65fbc282891cf0 (valor aleatorio utilizado: 15)
Commit de la parte 3: b1eb23eb3e399847098933f99e4ff7b049023d69096d6cec5f1a5f442124da9e (valor aleatorio utilizado: 14)
Commit de la parte 4: f5a40110dc4223a07ed8725c1d688d4a0722e019ef40d2d7f744a005f9ec501c (valor aleatorio utilizado: 5)
Commit de la parte 5: d6a807242950aee97965d11f902a9951acc3ea7425d3bf5d41c3dcc894aaadbc (valor aleatorio utilizado: 14)


## 2. Generación de los valores aleatorios r y $\epsilon$
##### (Verificador)

En el contexto de la prueba de cero conocimiento estos valores son generador por parte del verificador $V$, con la intención de corroborar si el probador puede responder a sus preguntas de manera correcta usando estos puntos. En este caso los vamos a generar de manera aleatoria representando la elección del verificador.

In [4]:
# Generar los valores aleatorios r y epsilon en el campo finito F_p
r = F_p.random_element()
epsilon = F_p.random_element()

# Imprimir los valores generados
print(f"Valor aleatorio r generado: {r}")
print(f"Valor aleatorio epsilon generado: {epsilon}")

Valor aleatorio r generado: 0
Valor aleatorio epsilon generado: 5


## 3. Ejecución del protocolo MPC en la "cabeza"
##### (Probador)

El siguiente paso es la ejecución del protocolo MPC por parte del probador haciendo uso de los valores aleatorios r y $\epsilon$ proporcionados por parte del verificador.

En este caso el protocolo MPC empieza desde la evaluación de los polinomios ya que los puntos aleatorios son creados en un paso previo.

In [5]:
# Implementación del protocolo MPC

# Paso 2: Evaluar los polinomios S, Q, y P en un punto aleatorio r del campo finito
def evaluar_polinomios_en_punto(polinomios, r):
    return [part(r) for part in polinomios]

print(f"Valor aleatorio {r}")
print(f"Valuación en punto aleatorio {S(r)*Q(r)==P(r)*F(r)}")

evaluaciones_S_en_r = evaluar_polinomios_en_punto(construir_polinomios_de_partes(shares_S, n_parts, X), r)
evaluaciones_Q_en_r = evaluar_polinomios_en_punto(construir_polinomios_de_partes(shares_Q, n_parts, X), r)
evaluaciones_P_en_r = evaluar_polinomios_en_punto(construir_polinomios_de_partes(shares_P, n_parts, X), r)

# Paso 3: Calcular [alpha] y [beta] para cada parte
def calcular_alpha_beta(epsilon, evaluaciones_Q, evaluaciones_S, a_shares, b_shares):
    comparticion_alpha = [epsilon * evaluaciones_Q[j] + a_shares[j] for j in range(n_parts)]
    comparticion_beta = [evaluaciones_S[j] + b_shares[j] for j in range(n_parts)]
    return comparticion_alpha, comparticion_beta

comparticion_alpha, comparticion_beta = calcular_alpha_beta(epsilon, evaluaciones_Q_en_r, evaluaciones_S_en_r, a_shares, b_shares)

# Paso 4: Transmitir [alpha] y [beta] para obtener alpha y beta
def reconstruir_valor(comparticiones):
    return sum(comparticiones)

alpha = reconstruir_valor(comparticion_alpha)
beta = reconstruir_valor(comparticion_beta)
print(f"\nValor de alpha: {alpha}")
print(f"Valor de beta: {beta}")

# Paso 4: Calcular [v] para cada parte
# Calcular c como la multiplicación de la suma de a_shares y la suma de b_shares, luego dividir c en partes aleatorias
def calcular_v(epsilon, evaluaciones_P, a_shares, b_shares, alpha, beta, c_shares):
    comparticion_v = []
    for j in range(n_parts):
        if j == 0:
            # Para el primer elemento, restamos alpha * beta
            v_j = (epsilon * F(r) * evaluaciones_P[j]) - c_shares[j] + alpha * b_shares[j] + beta * a_shares[j] - alpha * beta
        else:
            # Para los demás elementos, no restamos alpha * beta
            v_j = (epsilon * F(r) * evaluaciones_P[j]) - c_shares[j] + alpha * b_shares[j] + beta * a_shares[j]
        comparticion_v.append(v_j)
    return comparticion_v

comparticion_v = calcular_v(epsilon, evaluaciones_P_en_r, a_shares, b_shares, alpha, beta, c_shares)

# Paso 5: Transmitir [v] para obtener v
v = reconstruir_valor(comparticion_v)
print(f"\nValor de v: {v}")

# Paso 7: Verificar si v es igual a 0
def verificar_v(v):
    if v == 0:
        print("Las partes aceptan: v = 0")
    else:
        print(f"Las partes rechazan: v = {v}")

verificar_v(v)

Valor aleatorio 0
Valuación en punto aleatorio True

Valor de alpha: 6
Valor de beta: 0

Valor de v: 0
Las partes aceptan: v = 0


### Revelación de valores

Para que el verificador pueda corroborar la ejecución válida del protocolo y que cada parte tiene una salida exitosa, el probador revela los valores $[\alpha] , [\beta], [\nu]$ al verificador.

A partir de estos valores el verificador puede corroborar que la ejecución del protocolo es exitosa.

In [6]:
print(f"comparticion_alpha: {comparticion_alpha}")
print(f"comparticion_beta: {comparticion_beta}")
print(f"comparticion_v: {comparticion_v}")

comparticion_alpha: [3, 15, 13, 4, 5]
comparticion_beta: [1, 0, 8, 6, 2]
comparticion_v: [11, 11, 10, 16, 3]


## 4. Elección de $I$
##### (Verificador)

El verificador elige un subconjunto $I \subset [1 : N] $ de partes aleatorias para que el probador le revele las entradas y el verificador pueda simular la ejecución del protocolo para esas partes y verificar la consistencia con los compromisos iniciales así como la ejecución del protocolo.

In [18]:
# Verificador
numero_partes = 3  # Por ejemplo, queremos revelar 3 partes
indices_revelados = random.sample(range(n_parts), numero_partes)
print(f"\nÍndices de las partes seleccionadas para revelar: {indices_revelados}")


Índices de las partes seleccionadas para revelar: [4, 0, 2]


EL verificador le comparte estos índices al probador, y el probador revela las entradas originales de estas partes.

## 5. Envío de las entradas de cada parte elegida
##### (Probador)

A partir de las partes elegidas por el verificador, el probador revela las entradas de cada una de estas partes y se las envía al verificador.

El objetivo es que el verificador sea capaz de repetir la ejecución para estas partes y corroborar que la ejecución es la esperada, y que el probador no está mintiendo o intentando hacer una ejecución falsa.

In [22]:
# Probador
# Extraer e imprimir los valores de S, Q, P, a, b y c para los índices seleccionados
print("\nValores revelados para los índices seleccionados:")
shares_S_revelados = []
shares_Q_revelados = []
shares_P_revelados = []
a_shares_revelados = []
b_shares_revelados = []
c_shares_revelados = []
valores_aleatorios_revelados = []

for idx in indices_revelados:
    print(f"\nParte {idx + 1}:")
    print(f"  - Coeficientes de S: {shares_S[idx]}")
    print(f"  - Coeficientes de Q: {shares_Q[idx]}")
    print(f"  - Coeficientes de P: {shares_P[idx]}")
    print(f"  - Parte de a: {a_shares[idx]}")
    print(f"  - Parte de b: {b_shares[idx]}")
    print(f"  - Parte de c: {c_shares[idx]}")
    print(f"  - Elemento aleatorio del compromiso: {valores_aleatorios[idx]}")

    # Almacenar los valores revelados
    shares_S_revelados.append(shares_S[idx])
    shares_Q_revelados.append(shares_Q[idx])
    shares_P_revelados.append(shares_P[idx])
    a_shares_revelados.append(a_shares[idx])
    b_shares_revelados.append(b_shares[idx])
    c_shares_revelados.append(c_shares[idx])
    valores_aleatorios_revelados.append(valores_aleatorios[idx])


Valores revelados para los índices seleccionados:

Parte 5:
  - Coeficientes de S: [15, 11, 9, 10, 4, 5]
  - Coeficientes de Q: [5, 6, 4]
  - Coeficientes de P: [4, 15]
  - Parte de a: 14
  - Parte de b: 4
  - Parte de c: 4
  - Elemento aleatorio del compromiso: 14

Parte 1:
  - Coeficientes de S: [15, 16, 14, 13, 12, 12]
  - Coeficientes de Q: [7, 14, 1]
  - Coeficientes de P: [0, 0]
  - Parte de a: 2
  - Parte de b: 3
  - Parte de c: 7
  - Elemento aleatorio del compromiso: 12

Parte 3:
  - Coeficientes de S: [8, 3, 3, 14, 12, 1]
  - Coeficientes de Q: [14, 5, 15]
  - Coeficientes de P: [9, 16]
  - Parte de a: 11
  - Parte de b: 0
  - Parte de c: 7
  - Elemento aleatorio del compromiso: 14


Con los valores revelados por el probador, el verificador procede a verificar la ejecución del protocolo MPC para esas partes

## 6. Verificación consistencia
##### (Verificador)

Finalmente, el último paso es la verificación de la consistencia de la ejecución del protocolo MPC por parte del probador. Para esto el verificador repite la ejecución para las partes que eligió usando los valores suministrados por el probador. Esta etapa de verificación incluye la validación de los compromisos, esto con el objetivo de saber que sí eran los valores originales.

In [24]:
# Paso 1: Validar los compromisos con los valores recibidos
print("\nValidación de los compromisos para los índices seleccionados:")

compromisos_validos = True  # Inicializamos la bandera de compromiso válido

for idx in indices_revelados:
    # Valores revelados
    s = shares_S[idx]
    q = shares_Q[idx]
    p = shares_P[idx]
    a = a_shares[idx]
    b = b_shares[idx]
    c = c_shares[idx]
    valor_aleatorio = valores_aleatorios[idx]

    # Generar el compromiso utilizando los valores revelados
    datos_str = f"{s}{q}{p}{a}{b}{c}{valor_aleatorio}"
    compromiso_calculado = hashlib.sha256(datos_str.encode()).hexdigest()

    # Comparar con el compromiso original
    if compromiso_calculado != commits[idx]:
        print(f"Compromiso inválido para la parte {idx + 1}.")
        compromisos_validos = False
    else:
        print(f"Compromiso válido para la parte {idx + 1}.")

# Si la validación de los compromisos es exitosa, proceder con la comparación de valores
if compromisos_validos:
    print("\nCompromisos validados correctamente. Procediendo a la comparación de valores...")

    # Paso 2: Evaluar los polinomios S, Q, P en el punto aleatorio r usando los coeficientes revelados
    def evaluar_polinomio_en_punto(coeficientes_revelados, r):
        return sum(coef * (r ** i) for i, coef in enumerate(coeficientes_revelados))

    evaluaciones_S_revelados = []
    evaluaciones_Q_revelados = []
    evaluaciones_P_revelados = []

    for idx in range(len(indices_revelados)):
        evaluacion_S = evaluar_polinomio_en_punto(shares_S_revelados[idx], r)
        evaluacion_Q = evaluar_polinomio_en_punto(shares_Q_revelados[idx], r)
        evaluacion_P = evaluar_polinomio_en_punto(shares_P_revelados[idx], r)

        evaluaciones_S_revelados.append(evaluacion_S)
        evaluaciones_Q_revelados.append(evaluacion_Q)
        evaluaciones_P_revelados.append(evaluacion_P)

        print(f"\nParte {indices_revelados[idx] + 1}:")
        print(f"Evaluación de S en r: {evaluacion_S}")
        print(f"Evaluación de Q en r: {evaluacion_Q}")
        print(f"Evaluación de P en r: {evaluacion_P}")

    # Paso 3: Calcular [alpha] y [beta] para las partes seleccionadas y compararlos con los valores originales
    print("\nComparación de [alpha] y [beta] para los índices revelados:")

    comparacion_exitosa = True  # Inicializamos la bandera para verificar si todas las comparaciones son exitosas

    comparticion_alpha_original, comparticion_beta_original = calcular_alpha_beta_revelados(
        epsilon, evaluaciones_Q_revelados, evaluaciones_S_revelados, a_shares_revelados, b_shares_revelados
    )

    # Comparar los valores de alpha y beta en los índices revelados
    for idx in range(len(indices_revelados)):
        alpha_revelado = epsilon * evaluaciones_Q_revelados[idx] + a_shares_revelados[idx]
        beta_revelado = evaluaciones_S_revelados[idx] + b_shares_revelados[idx]

        alpha_original = comparticion_alpha_original[idx]
        beta_original = comparticion_beta_original[idx]

        print(f"\nParte {indices_revelados[idx] + 1}:")
        print(f"Alpha revelado: {alpha_revelado}, Alpha original: {alpha_original}")
        print("Alpha - Comparación: " + ("Iguales" if alpha_revelado == alpha_original else "Diferentes"))

        if alpha_revelado != alpha_original:
            comparacion_exitosa = False

        print(f"Beta revelado: {beta_revelado}, Beta original: {beta_original}")
        print("Beta - Comparación: " + ("Iguales" if beta_revelado == beta_original else "Diferentes"))

        if beta_revelado != beta_original:
            comparacion_exitosa = False

    # Paso 4: Calcular [v] para las partes seleccionadas y compararlos con los valores originales
    print("\nComparación de [v] para los índices revelados:")

    comparticion_v_original = calcular_v_revelados(epsilon, evaluaciones_P_revelados, a_shares_revelados, b_shares_revelados, alpha_revelado, beta_revelado, c_shares_revelados)

    for idx in range(len(indices_revelados)):
        if idx == 0:
            # Para el primer índice, restamos alpha_revelado * beta_revelado
            v_revelado = (epsilon * F(r) * evaluaciones_P_revelados[idx]) - c_shares_revelados[idx] + alpha_revelado * b_shares_revelados[idx] + beta_revelado * a_shares_revelados[idx] - alpha_revelado * beta_revelado
        else:
            # Para los demás índices, no restamos alpha_revelado * beta_revelado
            v_revelado = (epsilon * F(r) * evaluaciones_P_revelados[idx]) - c_shares_revelados[idx] + alpha_revelado * b_shares_revelados[idx] + beta_revelado * a_shares_revelados[idx]

        v_original = comparticion_v_original[idx]

        print(f"\nParte {indices_revelados[idx] + 1}:")
        print(f"v revelado: {v_revelado}, v original: {v_original}")
        print("v - Comparación: " + ("Iguales" if v_revelado == v_original else "Diferentes"))

        if v_revelado != v_original:
            comparacion_exitosa = False

    # Verificación final
    if comparacion_exitosa:
        print("\nResultado final: Aceptado")
    else:
        print("\nResultado final: Rechazado")

else:
    print("\nResultado final: Rechazado debido a compromisos inválidos")


Validación de los compromisos para los índices seleccionados:
Compromiso válido para la parte 5.
Compromiso válido para la parte 1.
Compromiso válido para la parte 3.

Compromisos validados correctamente. Procediendo a la comparación de valores...

Parte 5:
Evaluación de S en r: 15
Evaluación de Q en r: 5
Evaluación de P en r: 4

Parte 1:
Evaluación de S en r: 15
Evaluación de Q en r: 7
Evaluación de P en r: 0

Parte 3:
Evaluación de S en r: 8
Evaluación de Q en r: 14
Evaluación de P en r: 9

Comparación de [alpha] y [beta] para los índices revelados:

Parte 5:
Alpha revelado: 5, Alpha original: 5
Alpha - Comparación: Iguales
Beta revelado: 2, Beta original: 2
Beta - Comparación: Iguales

Parte 1:
Alpha revelado: 3, Alpha original: 3
Alpha - Comparación: Iguales
Beta revelado: 1, Beta original: 1
Beta - Comparación: Iguales

Parte 3:
Alpha revelado: 13, Alpha original: 13
Alpha - Comparación: Iguales
Beta revelado: 8, Beta original: 8
Beta - Comparación: Iguales

Comparación de [v] pa